In [58]:
#first, we'll install nltk 
!pip install nltk

In [59]:
#let's import important libraries 
import pandas as pd
import numpy as np
import re 
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [60]:
#we'll download stopwords from the library
import nltk 
nltk.download ('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [61]:
#let's print stopwords in English 
print(stopwords.words ('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

### Data Preprossing

In [62]:
#let's load the dataset into pandas dataframe 
news_df=pd.read_csv('train.csv')

In [63]:
#we'll view the first 5 rows on the dataframe 
news_df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


Details about the dataset:
- id: unique id for a news article
-  title: the title of a news article
-   author: author of the news article
-    text: the text of the articles, this could be incomplete
-    label: a label makes whether the news article is real or fake --> 1 - fake news, while 0-real news

In [64]:
#let's view the number of rows and columns we have in the dataset
news_df.shape

(20800, 5)

In [65]:
#let's check for missing values in the dataset
news_df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

- we'll drop the missing values, since we have large dataset ( if the numbers of missing values are less than 5% of the dataset, we'll drop the missing values)

In [66]:
#we'll replace the null values with empty string
news_df=news_df.fillna('')

In [67]:
#let's check if changes is effected
news_df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

- we'll need just the title and author columns, because the text column are so large ( this will take a lot of time to process) 

In [68]:
#merging the authors' name and news title 
news_df['content']=news_df['author']+ '' +news_df['title']

In [69]:
#let's print our dataset
print(news_df['content'])

0        Darrell LucusHouse Dem Aide: We Didn’t Even Se...
1        Daniel J. FlynnFLYNN: Hillary Clinton, Big Wom...
2        Consortiumnews.comWhy the Truth Might Get You ...
3        Jessica Purkiss15 Civilians Killed In Single U...
4        Howard PortnoyIranian woman jailed for fiction...
                               ...                        
20795    Jerome HudsonRapper T.I.: Trump a ’Poster Chil...
20796    Benjamin HoffmanN.F.L. Playoffs: Schedule, Mat...
20797    Michael J. de la Merced and Rachel AbramsMacy’...
20798    Alex AnsaryNATO, Russia To Hold Parallel Exerc...
20799               David SwansonWhat Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [70]:
news_df.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell LucusHouse Dem Aide: We Didn’t Even Se...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. FlynnFLYNN: Hillary Clinton, Big Wom..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.comWhy the Truth Might Get You ...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss15 Civilians Killed In Single U...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard PortnoyIranian woman jailed for fiction...


#we'll separate the data from label
X=news_df.drop(columns='label', axis=1)
y=news_df['label']

print(X)
print(y)

- Stemming procedure: this is the process the words to its root word
  example: actor, actions, acting --> act

In [84]:
port_stem=PorterStemmer()

In [85]:
def stemming(text):
    stemmed_text = re.sub('[^a-zA-Z]',' ',text)
    stemmed_text = stemmed_text.lower()
    stemmed_text = stemmed_text.split()
    stemmed_text = [port_stem.stem(word) for word in stemmed_text if word not in stopwords.words('english')]
    stemmed_text = ' '.join(stemmed_text)
    return stemmed_text

In [86]:
#let's apply the function to the content column
news_df['content']=news_df['content'].apply(stemming)

In [87]:
#let's print the content dataset after cleaning 
print(news_df['content'])

0        darrel lucushous dem aid even see comey letter...
1        daniel j flynnflynn hillari clinton big woman ...
2                consortiumnew comwhi truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoyiranian woman jail fiction unpub...
                               ...                        
20795    jerom hudsonrapp trump poster child white supr...
20796    benjamin hoffmann f l playoff schedul matchup ...
20797    michael j de la merc rachel abramsmaci said re...
20798    alex ansarynato russia hold parallel exercis b...
20799                        david swansonwhat keep f aliv
Name: content, Length: 20800, dtype: object


In [88]:
#next, we'll separate the data from the label
X=news_df['content'].values
y=news_df['label'].values

In [89]:
#we'll convert the textual data to numerical data
vectorizer=TfidfVectorizer()
vectorizer.fit(X)

X=vectorizer.transform(X)
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 198373 stored elements and shape (20800, 28419)>
  Coords	Values
  (0, 578)	0.2694167078545384
  (0, 4211)	0.36253203231506576
  (0, 5006)	0.24725958235728157
  (0, 5969)	0.35488202138141456
  (0, 6273)	0.2839932825877812
  (0, 8022)	0.2313366174248873
  (0, 12782)	0.24619727512767192
  (0, 14555)	0.2917725968420029
  (0, 15019)	0.4300622675963931
  (0, 22724)	0.25523360180691607
  (0, 26340)	0.2808837940159642
  (1, 2622)	0.3562953366945267
  (1, 3281)	0.18652439327549428
  (1, 3859)	0.45980466668763476
  (1, 4767)	0.23338756776626793
  (1, 5916)	0.31810058109638056
  (1, 8772)	0.5258635625386451
  (1, 11313)	0.24166773097712638
  (1, 27923)	0.36911845953845024
  (2, 5121)	0.5511414848555652
  (2, 5240)	0.40440534260277944
  (2, 8567)	0.3411947414020896
  (2, 9454)	0.30743020569262086
  (2, 16361)	0.43295215406038445
  (2, 26235)	0.3665032495181434
  :	:
  (20797, 1249)	0.3072223353708335
  (20797, 2257)	0.3357782642976524


In [90]:
#splitting the dataset to training and test dataset
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, stratify=y, random_state=2)

## Training the model

In [91]:
#we'll train our model using LoLogisticRegression, which is best for binary classification
model=LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [92]:
#evaluation (using accuracy score)
#accuracy_score on the training data 
X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(X_train_prediction, y_train)

print('Accuracy score of training data: ', training_data_accuracy)

Accuracy score of training data:  0.971875


In [93]:
#accuracy_score on the test data 
X_test_prediction=model.predict(X_test)
test_data_accuracy=accuracy_score(X_test_prediction, y_test)

print('Accuracy score of test data: ', test_data_accuracy)

Accuracy score of test data:  0.9550480769230769


In [95]:
# Save model and vectorizer
import pickle
with open('fake_news_model.pkl', 'wb') as file:
    pickle.dump(model, file)